# AutoML: Build Production-Ready Models Quickly!

https://www.linkedin.com/learning/automl-build-production-ready-models-quickly/

In [1]:
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.core.metrics import make_scorer
from sklearn.model_selection import train_test_split

In [2]:
# train_data = TabularDataset('../01-machine-learning-recap/data/train_data.csv')
train_data = TabularDataset('train_data_insure.csv')

In [3]:
train_data.head()

,Customer Id,YearOfObservation,Insured_Period,Residential,Building_Painted,Building_Fenced,Garden,Settlement,Building Dimension,Building_Type,Date_of_Occupancy,NumberOfWindows,Geo_Code,Claim
0,H14663,2013,1.0,0,N,V,V,U,290.0,1,1960.0,.,1053,0
1,H2037,2015,1.0,0,V,N,O,R,490.0,1,1850.0,4,1053,0
2,H3802,2014,1.0,0,N,V,V,U,595.0,1,1960.0,.,1053,0
3,H3834,2013,1.0,0,V,V,V,U,2840.0,1,1960.0,.,1053,0
4,H5053,2014,1.0,0,V,N,O,R,680.0,1,1800.0,3,1053,0


In [4]:
train, test = train_test_split(train_data, test_size=0.3, shuffle=True)

In [5]:
EVAL_METRIC = "f1"
SAVE_PATH = "AutoGluonModels_improved"   # Trained models will be saved here
LABEL = "Claim"

## Improving your AutoGluon Model

### 1. Via Feature Engineering

In [6]:
train["age_of_building"] = train["YearOfObservation"] - train["Date_of_Occupancy"]
test["age_of_building"] = test["YearOfObservation"] - test["Date_of_Occupancy"]
train["YearOfObservation"] = train["YearOfObservation"].astype("category")
test["YearOfObservation"] = test["YearOfObservation"].astype("category")

### 2. Use the Right Preset & refit_full

In [7]:
predictor = TabularPredictor(label=LABEL, path=SAVE_PATH, eval_metric=EVAL_METRIC)
predictor = predictor.fit(train, presets=['best_quality'], time_limit=100, refit_full='best')

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 100s
AutoGluon will save models to "AutoGluonModels_improved/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu May 4 15:21:22 UTC 2023
Train Data Rows:    5012
Train Data Columns: 14
Label Column: Claim
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:   

/home/stever7/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/home/stever7/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/home/stever7/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/home/stever7/.local/lib/pyt

In [8]:
predictor.leaderboard(extra_info=True, silent=True).head()

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order,num_features,...,hyperparameters,hyperparameters_fit,ag_args_fit,features,compile_time,child_hyperparameters,child_hyperparameters_fit,child_ag_args_fit,ancestors,descendants
0,WeightedEnsemble_L2,0.385842,0.331863,24.537182,0.012880,4.327517,2,True,12,2,...,"{'use_orig_features': False, 'max_base_models': 25, 'max_base_models_per_type': 5, 'save_bag_folds': True}",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None, 'drop_unique': False}","[NeuralNetFastAI_BAG_L1, LightGBMXT_BAG_L1]",None,{'ensemble_size': 100},{'ensemble_size': 17},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None, 'drop_unique': False}","[NeuralNetFastAI_BAG_L1, LightGBMXT_BAG_L1]",[]
1,NeuralNetFastAI_BAG_L1,0.385602,0.209237,13.434163,0.209237,13.434163,1,True,10,13,...,"{'use_orig_features': True, 'max_base_models': 25, 'max_base_models_per_type': 5, 'save_bag_folds': True}",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None, 'drop_unique': False}","[Residential, Settlement, YearOfObservation, Building_Painted, Building_Type, Date_of_Occupancy, NumberOfWindows, Insured_Period, Building_Fenced, Garden, Geo_Code, Building Dimension, age_of_building]",None,"{'layers': None, 'emb_drop': 0.1, 'ps': 0.1, 'bs': 'auto', 'lr': 0.01, 'epochs': 'auto', 'early.stopping.min_delta': 0.0001, 'early.stopping.patience': 20, 'smoothing': 0.0}","{'epochs': 30, 'best_epoch': 9}","{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': ['bool', 'int', 'float', 'category'], 'valid_special_types': None, 'ignored_type_group_special': ['text_ngram', 'text_as_category'], 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None}",[],[WeightedEnsemble_L2]
2,LightGBM_BAG_L1,0.365923,0.110091,7.580386,0.110091,7.580386,1,True,4,13,...,"{'use_orig_features': True, 'max_base_models': 25, 'max_base_models_per_type': 5, 'save_bag_folds': True}",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None, 'drop_unique': False}","[Residential, Settlement, YearOfObservation, Building_Painted, Building_Type, Date_of_Occupancy, NumberOfWindows, Insured_Period, Building_Fenced, Garden, Geo_Code, Building Dimension, age_of_building]",None,{'learning_rate': 0.05},{'num_boost_round': 446},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': ['bool', 'int', 'float', 'category'], 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_s

### 3. Via CustomMetrics

In [9]:
fp_cost = 1000  # Admin fees spent to investigate potential insurance claim
fn_cost = 10000 # Average insurance claim made
tp_cost = 1000 # Admin fees spent to investigate potential insurance claim
tn_cost = 0

In [10]:
def cost_fn(y_true, y_prob):
    tp = np.where((y_prob >= 0.7) & (y_true==1), tp_cost, 0)
    fp = np.where((y_prob >= 0.7) & (y_true==0), fp_cost, 0)
    tn = np.where((y_prob < 0.7) & (y_true==0), tn_cost, 0)
    fn = np.where((y_prob < 0.7) & (y_true==1), fn_cost, 0)
    return np.sum([tp,fp,tn,fn])

In [11]:
my_scorer = make_scorer(
    name="operating_cost",
    score_func=cost_fn,
    greater_is_better=False,
    needs_proba=True
)

In [12]:
predictor.leaderboard(test, extra_metrics=[my_scorer], silent=True).head()

,model,score_test,operating_cost,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1,0.376694,-4308000,0.365923,0.143363,0.110091,7.580386,0.143363,0.110091,7.580386,1,True,4
1,XGBoost_BAG_L1,0.363636,-4385000,0.345977,0.310239,0.172047,8.703592,0.310239,0.172047,8.703592,1,True,11
2,LightGBMXT_BAG_L1,0.362869,-4374000,0.350797,0.161988,0.109746,6.775502,0.161988,0.109746,6.775502,1,True,3
3,WeightedEnsemble_L2,0.362468,-4244000,0.385842,0.574867,0.331863,24.537182,0.004168,0.012880,4.327517,2,True,12
4,NeuralNetFastAI_BAG_L1,0.360153,-4214000,0.385602,0.408712,0.209237,13.434163,0.408712,0.209237,13.434163,1,True,10
